A simple Python implementation of the core 'budgeting boxes' pairwise voting algorithm used for allocating token rewards and funding, as described in this whitepaper:
https://uploads-ssl.webflow.com/61840fafb9a4c433c1470856/639b50ee30b729cb016806c1_BudgetingBoxes.pdf

This alternate form of governance voting is designed to mitigate many of the shortcomings of the current "1 token, 1 vote, >50% wins" status quo for decentralized governance:
1. Spamming the mechanism with fake projects
2. Scammers claiming to represent legitimate projects
3. Projects bribing voters to vote for projects in excess of their true value
4. Winning projects using their reputation to vote themselves up
5. Winning projects colluding to claim a larger share of CLNY
6. Voter apathy
7. Cognitive biases of various kinds, like the Keynesian Beauty Contest
8. Voters voting dishonestly or randomly
9. Sybil attacks of any sort (either among voters or projects)

In [1]:
import numpy as np

In [2]:
# Conduct Pairwise Voting Process

'''
a,b,c,d

a vs b A wins
a vs c C wins
a vs d A wins

b vs a A wins
b vs c B wins
b vs d B wins

c vs a C wins
c vs b B wins
c vs d D wins

d vs a A wins
d vs b B wins
d vs c D wins

'''

'\na,b,c,d\n\na vs b A wins\na vs c C wins\na vs d A wins\n\nb vs a A wins\nb vs c B wins\nb vs d B wins\n\nc vs a C wins\nc vs b B wins\nc vs d D wins\n\nd vs a A wins\nd vs b B wins\nd vs c D wins\n\n'

In [3]:
# Represent Pairwise Voting Results As an Array of Wins

M = np.array([
    [2,1,0,1],
    [0,2,1,1],
    [1,0,1,0],
    [0,0,1,1]
])

In [4]:
# Transform M into Probabilities

column_sums = M.sum(axis=0)
M = M / column_sums
M

array([[0.66666667, 0.33333333, 0.        , 0.33333333],
       [0.        , 0.66666667, 0.33333333, 0.33333333],
       [0.33333333, 0.        , 0.33333333, 0.        ],
       [0.        , 0.        , 0.33333333, 0.33333333]])

In [5]:
# Add Dampening Factor

def dampening(M, K, d):
    M = (d*M) + ((1-d) / K)
    return M

M = dampening(M,4,1)
M

array([[0.66666667, 0.33333333, 0.        , 0.33333333],
       [0.        , 0.66666667, 0.33333333, 0.33333333],
       [0.33333333, 0.        , 0.33333333, 0.        ],
       [0.        , 0.        , 0.33333333, 0.33333333]])

In [6]:
# Compute the Eigenvector `v` Representing Budget or Ranking

eps = 0.001 # Tolerance for convergence

def ranking_eigenvector(M, eps):
    n_rows, n_cols = M.shape
    
    if n_rows != n_cols:
        raise ValueError('Requires an nxn matrix. M provided is not of expected dimensions.')
    
    v = np.ones(n_rows) / np.sqrt(n_rows)  # Initialize v to a normalized vector
    while True:
        Mv = np.dot(M, v)
        Mv = Mv / np.linalg.norm(Mv)  # Normalize the result
        if np.linalg.norm(v - Mv) < eps:  # Check for convergence
            break
        v = Mv
        
    v_normalized = v / np.sum(v) # Normalize to sum to 1
    
    # Determine number of decimal places in `eps`
    str_eps = str(eps)
    decimal_index = str_eps.find('.')
    if decimal_index != -1:
        decimals = len(str_eps) - decimal_index - 1
    else:
        decimals = 0
        
    v_rounded = np.round(v_normalized, decimals) # Round each item to `eps` number of decimals
    
    return v_rounded

ranking_eigenvector(M, eps)

array([0.4, 0.3, 0.2, 0.1])